# Import Libraries

In [1]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype

from datetime import datetime

import matplotlib.pyplot as plt
import seaborn as sns


sns.set()

%matplotlib inline
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

# Read the Data

## About the Data

The dataset I'm using is from [Kaggle](https://www.kaggle.com/dcohen21/8anu-climbing-logbook). It was collected from the website [8a.nu](https://www.8a.nu/) by David Cohen. This dataset was collected on 9/13/2017.

The idea behind [8a.nu](https://www.8a.nu/) is to provide a logbook for climbers to keep track of their climbs digitally. Many climbers have a manual logbook that they use to keep track of this information, and [8a.nu](https://www.8a.nu/) provides a digital alternative to this. Fortunately for us this means we have a wealth of information available.

## Import the Data

The download from [Kaggle](https://www.kaggle.com/dcohen21/8anu-climbing-logbook) is in sqlite format, which is nice, but pandas seemed to read this information quite slowly so I converted each of the tables within the database into csv's from the sqlite3 interface. Pandas reads the csv files much faster.


In [2]:
input_path = '../data/input/'
ascents = pd.read_csv(input_path + 'ascent.csv', low_memory=False)
grades = pd.read_csv(input_path + 'grade.csv', low_memory=False)
method = pd.read_csv(input_path + 'method.csv', low_memory=False)
user = pd.read_csv(input_path + 'user.csv', low_memory=False)

In [3]:
print(ascents.info(verbose = True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4111877 entries, 0 to 4111876
Data columns (total 28 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   id                    int64 
 1   user_id               int64 
 2   grade_id              int64 
 3   notes                 object
 4   raw_notes             int64 
 5   method_id             int64 
 6   climb_type            int64 
 7   total_score           int64 
 8   date                  int64 
 9   year                  int64 
 10  last_year             int64 
 11  rec_date              int64 
 12  project_ascent_date   int64 
 13  name                  object
 14  crag_id               int64 
 15  crag                  object
 16  sector_id             int64 
 17  sector                object
 18  country               object
 19  comment               object
 20  rating                int64 
 21  description           object
 22  yellow_id             int64 
 23  climb_try             int64 
 24

In [4]:
print(grades.info(verbose = True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83 entries, 0 to 82
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     83 non-null     int64 
 1   score                  83 non-null     int64 
 2   fra_routes             83 non-null     object
 3   fra_routes_input       83 non-null     int64 
 4   fra_routes_selector    83 non-null     int64 
 5   fra_boulders           83 non-null     object
 6   fra_boulders_input     83 non-null     int64 
 7   fra_boulders_selector  83 non-null     int64 
 8   usa_routes             47 non-null     object
 9   usa_routes_input       83 non-null     int64 
 10  usa_routes_selector    83 non-null     int64 
 11  usa_boulders           45 non-null     object
 12  usa_boulders_input     83 non-null     int64 
 13  usa_boulders_selector  83 non-null     int64 
dtypes: int64(10), object(4)
memory usage: 9.2+ KB
None


In [5]:
print(method.info(verbose = True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         5 non-null      int64 
 1   score      5 non-null      int64 
 2   shorthand  5 non-null      object
 3   name       5 non-null      object
dtypes: int64(2), object(2)
memory usage: 288.0+ bytes
None


In [6]:
print(user.info(verbose = True))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62593 entries, 0 to 62592
Data columns (total 22 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            62593 non-null  int64 
 1   first_name    62593 non-null  object
 2   last_name     62593 non-null  object
 3   city          61163 non-null  object
 4   country       62495 non-null  object
 5   sex           62593 non-null  int64 
 6   height        62593 non-null  int64 
 7   weight        62593 non-null  int64 
 8   started       62593 non-null  int64 
 9   competitions  4888 non-null   object
 10  occupation    11778 non-null  object
 11  sponsor1      5355 non-null   object
 12  sponsor2      2656 non-null   object
 13  sponsor3      1497 non-null   object
 14  best_area     15413 non-null  object
 15  worst_area    220 non-null    object
 16  guide_area    7755 non-null   object
 17  interests     8577 non-null   object
 18  birth         27856 non-null  object
 19  pres

We can look at the climb type to see the split between climbs and boulders within the database:<br>
0 = climb<br>
1 = boulder<br>

In [7]:
ascents.climb_type.value_counts()

0    2875675
1    1236202
Name: climb_type, dtype: int64

It's interesting to note the number of registered users versus the number of user that have logged at least one climb.

In [8]:
unique_climbers = (pd.merge(left = ascents, left_on = 'user_id',
                            right = user, right_on = 'id',
                            how = 'left'
                           ))
print('climbers registered = {}\nclimbers logged = {}'
      .format(user.id.nunique(), unique_climbers.user_id.nunique()))

climbers registered = 62593
climbers logged = 36034


### User Table

In [9]:
user.head(2)

,id,first_name,last_name,city,country,sex,height,weight,started,competitions,occupation,sponsor1,sponsor2,sponsor3,best_area,worst_area,guide_area,interests,birth,presentation,deactivated,anonymous
0,1,first,last,Göteborg,SWE,0,177,73,1996,NaN,NaN,NaN,NaN,NaN,"Railay beach, Krabi, Thailand",NaN,NaN,NaN,1976-03-10,NaN,0,0
1,2,first,last,stockholm,SWE,0,0,0,2000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


What units are `sex`, `height`, and `weight`?

Previously, I determined that Adam Ondra had a `user_id` = 1476, so I can use this to reverse engineer the units.

In [10]:
user[user.id == 1476]

,id,first_name,last_name,city,country,sex,height,weight,started,competitions,occupation,sponsor1,sponsor2,sponsor3,best_area,worst_area,guide_area,interests,birth,presentation,deactivated,anonymous
1422,1476,first,last,Brno,CZE,0,185,68,1999,World cup lead and bouldering winner,student of Economics,Montura,Black Diamond,La Sportiva,"Flatanger, anywhere in Spain, Gemona",NaN,Moravsky kras,snowboarding,1993-02-05,my other sponsors:\r\nTendon www.mytendon.cz\r...,0,0


From this, I can see the following:<br>
- `sex`: 0 = Male
- `height`: cm
- `weight`: kg

Also, looking at this list of fields, I think it's pretty clear that I don't want very many of them. For now I will do a fairly large filter, and I guess I can always change this later.

In [11]:
user.loc[:,'sex'] = user.sex.replace({0:'Male', 1:'Female'}).astype('category')
user_cols = ['id', 'city', 'country', 'sex', 'height', 'weight', 'started', 'birth', 'occupation', 'sponsor1', 'deactivated']
user_filtered = user[user_cols].copy()
f = (user_filtered.birth > '1930-01-01') & (user_filtered.deactivated == 0)
user_filtered = user_filtered[f].drop(columns='deactivated').copy()
user_filtered.loc[:,'birth'] = pd.to_datetime(user_filtered.birth)
user_filtered.head()

,id,city,country,sex,height,weight,started,birth,occupation,sponsor1
0,1,Göteborg,SWE,Male,177,73,1996,1976-03-10,NaN,NaN
2,3,Umeå,SWE,Male,180,78,1995,1973-09-09,NaN,NaN
3,4,Goteborg,SWE,Female,165,58,2001,1984-07-26,NaN,NaN
4,5,North Attleboro,USA,Male,0,0,1991,1969-05-07,NaN,NaN
5,6,Sydney,AUS,Male,185,73,1992,1970-12-02,NaN,NaN


Let's do a little more filtering:
* Limit height between 122cm and 207cm (4'0" to 6'8")
* Weight > 0
* Add a BMI calculation
* Limit BMI between 12-40
* Remove anyone without a sex



In [12]:
# create the first filter
#  might as well filter on heigh and weight before calculating BMI
f = (user_filtered.sex.isin(['Male', 'Female']) &
     (user_filtered.weight > 0) &
     (user_filtered.height > 122) & 
     (user_filtered.height < 213) &
     (~user_filtered.country.isna())
    )
user_filtered = user_filtered[f].copy()

# calculate BMI and then filter on it as well
user_filtered['bmi'] = user_filtered.weight / (user_filtered.height/100)**2
f = (user_filtered.bmi > 12) & (user_filtered.bmi < 40)
user_filtered = user_filtered[f].copy()

# calculate age
date_collected = datetime.strptime('9/13/2017', '%m/%d/%Y') # date that the data was collected
user_filtered['current_age'] = (date_collected - user_filtered.birth).astype('timedelta64[Y]').astype('int64')

print('climbers remaining registered = {}'.format(user_filtered.id.nunique()))

climbers remaining registered = 20203


That gives us a decent picture of the users. We can see that through our filtering we have reduced the number of unique users from 62593 to 20203.

### Ascent Table

Ascent table is the biggest, and will require the most ammount of work. Let's begin.

In [13]:
ascents.head(2)

,id,user_id,grade_id,notes,raw_notes,method_id,climb_type,total_score,date,year,last_year,rec_date,project_ascent_date,name,crag_id,crag,sector_id,sector,country,comment,rating,description,yellow_id,climb_try,repeat,exclude_from_ranking,user_recommended,chipped
0,2,1,36,NaN,0,3,0,545,918342000,1999,0,1107126000,0,The King And I,16596,Railay,61,Dum's kitchen,THA,Thailand\n,0,NaN,255,0,0,0,0,0
1,3,1,36,NaN,0,3,0,545,925509600,1999,0,1107126000,0,vet ej,0,Nya berg - segl.,0,NaN,NaN,\n,0,NaN,0,0,0,0,0,0


First I want to fix a few grading issues at the upper end of the scale. The main purpose of this is fix "Silence" as I will probably talk about it and I want it to be by itself as it is the hardest climb in the world.

In [14]:
ascents[ascents.grade_id > 80].sort_values(by='name')
ascents.loc[((ascents.name == 'Chilam Balam') & (ascents.crag == 'Villanueva del Rosario')) , 'grade_id'] = 80

f = (((ascents.name == 'La Dura dura') & (ascents.crag == 'Oliana')) |
     ((ascents.name.isin(['Change', 'The Change'])) & (ascents.crag == 'Flatanger')) |
     ((ascents.name == 'Change') & (ascents.crag == 'Sloup')))

ascents.loc[f, 'grade_id'] = 83
ascents.loc[(ascents.name == 'Silence') & (ascents.crag == 'Flatanger') , 'grade_id'] = 85

# Let's convert the dates from unix style to datetime objects.
ascents.loc[:, 'date'] = (pd
                          .to_datetime(ascents.date, unit = 's')
                          .dt.normalize()
                          .sort_values())
ascents.loc[:, 'rec_date'] = (pd
                              .to_datetime(ascents.rec_date, unit = 's')
                              .dt.normalize()
                              .sort_values())

# get rid of unused columns
asecents_drop_cols = ['chipped', 'user_recommended', 'exclude_from_ranking', 
                      'repeat', 'climb_try', 'description', 'rating', 'comment', 
                      'sector_id', 'yellow_id', 'crag_id', 'notes', 'raw_notes', 'total_score', 
                      'project_ascent_date', 'last_year', 'rec_date']
ascents_filtered = ascents.drop(columns = asecents_drop_cols)

Now we can split the ascents file into boulders and roped climbs, which is what I am going to be analyzing in this project.<br>
- climb_type = 1 = boulder
- climb_type = 0 = roped climb

In [15]:
boulders = ascents_filtered[ascents_filtered.climb_type == 1].copy()
routes = ascents_filtered[ascents_filtered.climb_type == 0].copy()

print("Ropes info - take a quick look at the dataframe.")
print(routes.info(verbose=True, null_counts=True))

print("\nLook at the null values")
print(routes.isnull().sum())

print("\nBoulders info - take a quick look at the dataframe.")
print(boulders.info(verbose=True, null_counts=True))

Ropes info - take a quick look at the dataframe.
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2875675 entries, 0 to 4111875
Data columns (total 11 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   id          2875675 non-null  int64         
 1   user_id     2875675 non-null  int64         
 2   grade_id    2875675 non-null  int64         
 3   method_id   2875675 non-null  int64         
 4   climb_type  2875675 non-null  int64         
 5   date        2875675 non-null  datetime64[ns]
 6   year        2875675 non-null  int64         
 7   name        2875421 non-null  object        
 8   crag        2870319 non-null  object        
 9   sector      2137532 non-null  object        
 10  country     2818840 non-null  object        
dtypes: datetime64[ns](1), int64(6), object(4)
memory usage: 263.3+ MB
None

Look at the null values
id                 0
user_id            0
grade_id           0
method_id          0
cli

That actually looks reasonable. If I need to deal with the missing values later I can do that.

### Grades Table

Grades actually might require the most cleaning, but it is small so it shouldn't be hard. I want to deal with the Yosemite scale ratings rather than the French ratings, as that is the rating system that we use in Canada.

In [16]:
grades.loc[:,['usa_routes', 'usa_boulders']] = grades.loc[:,['usa_routes', 'usa_boulders']].fillna(method = 'ffill')

# fix some specific issues with the USA routes column
grades.loc[grades.fra_routes == '9c+', 'usa_routes'] = '5.15d'
grades.loc[grades.fra_routes == '9c+/10a', 'usa_routes'] = '5.15d'

grades.loc[:,'usa_routes'] = grades.loc[:,'usa_routes'].replace({'5.14b/c':'5.14b',
                                                                 '5.14c/d':'5.14c',
                                                                 '5.14d/.15a':'5.14d',
                                                                 '5.15a/b':'5.15a',
                                                                 '5.15b/c':'5.15b'})

# fix some specific issues with the USA boulders column
grades.loc[grades.fra_routes == '9c+', 'usa_boulders'] = 'V21'
grades.loc[grades.fra_routes == '9c+/10a', 'usa_boulders'] = 'V21'

grades.loc[:,'usa_boulders'] = grades.loc[:,'usa_boulders'].replace({'V3/4':'V3',
                                                                     'V4/5':'V4',
                                                                     'V5/6':'V5',
                                                                     'V8/9':'V8',
                                                                     'V15/16':'V15',
                                                                     'V16/17':'V16',
                                                                     'V17/18':'V17',
                                                                     'V18/19':'V18',
                                                                     'V19/20':'V19'})
# drop columns that we won't be using
grades_drop_cols = ['fra_routes_input', 'fra_routes_selector', 'fra_boulders_input', 'fra_boulders_selector',
                    'usa_routes_input', 'usa_routes_selector', 'usa_boulders_input', 'usa_boulders_selector']
grades_filtered = grades.drop(columns = grades_drop_cols)

# convert the two USA clibing grades to categorical types
usa_routes_type = CategoricalDtype(categories = list(grades_filtered.usa_routes.unique()),
                                   ordered = True)
usa_boulders_type = CategoricalDtype(categories = list(grades_filtered.usa_boulders.unique()),
                                   ordered = True)
grades_filtered['usa_routes'] = grades_filtered.usa_routes.astype(usa_routes_type)
grades_filtered['usa_boulders'] = grades_filtered.usa_boulders.astype(usa_boulders_type)

## Time to Merge

In [17]:
routes_final = pd.merge(left = routes, left_on = 'user_id',
                        right = user_filtered, right_on = 'id')
routes_final = pd.merge(left = routes_final, left_on = 'grade_id',
                        right = grades_filtered, right_on = 'id')

route_drop_cols = ['id_x', 'climb_type', 'sector', 'id_y', 'id', 'score']
routes_final = (routes_final.drop(columns = route_drop_cols)
                            .rename(columns = {'country_x':'country_route', 'country_y':'country_climber'}))
routes_final.loc[:,'birth'] = pd.to_datetime(routes_final.birth)

# add a column for the route codes as these can be used to sort by later
routes_final['usa_routes_codes'] = routes_final.usa_routes.cat.codes

# add an age at climb
#    there are some problems with this calculation as, for wahtever reason, people have input the data wrong
#    if this gets used in analysis then we will have to resolve the problems then
routes_final['age_at_climb'] = (routes_final.date - routes_final.birth).astype('timedelta64[Y]').astype('int64')

In [18]:
print(routes_final.isna().sum())

user_id                   0
grade_id                  0
method_id                 0
date                      0
year                      0
name                    155
crag                   2294
country_route         31957
city                  30663
country_climber           0
sex                       0
height                    0
weight                    0
started                   0
birth                     0
occupation          1023537
sponsor1            1258758
bmi                       0
current_age               0
fra_routes                0
fra_boulders              0
usa_routes                0
usa_boulders              0
usa_routes_codes          0
age_at_climb              0
dtype: int64


At this point we are missing a few values in `name`, `crag`, `country_route`, `city`. We might as well do a little more cleaning and just tidy these up as we don't lose a lot of information by doing so and we end up with a much more complete data set.

In [19]:
routes_final_clean = routes_final[(~routes_final.country_route.isna()) &
                                  (~routes_final.name.isna()) &
                                  (~routes_final.city.isna()) &
                                  (~routes_final.crag.isna())].copy()

In [20]:
routes_final_clean.isna().sum()

user_id                   0
grade_id                  0
method_id                 0
date                      0
year                      0
name                      0
crag                      0
country_route             0
city                      0
country_climber           0
sex                       0
height                    0
weight                    0
started                   0
birth                     0
occupation           988376
sponsor1            1213284
bmi                       0
current_age               0
fra_routes                0
fra_boulders              0
usa_routes                0
usa_boulders              0
usa_routes_codes          0
age_at_climb              0
dtype: int64

In [21]:
print('climbers logged = {}'.format(routes_final_clean.user_id.nunique()))

climbers logged = 13773


Finally, we can write these cleaned dataframes out to csv files for analysis.

In [22]:
output_path = '../data/cleaned/'
routes_final_clean.to_csv(output_path + 'climbing_data.csv')
user_filtered.to_csv(output_path + 'users_cleaned.csv')
grades_filtered.to_csv(output_path + 'grades_cleaned.csv')